In [1]:
import pandas as pd
import numpy as np
from calibration_tools import *

In [2]:
tenHighFidelityStarsNearby = [148280422016753413,148620420374152983,148620420599947827,148620420902559763,
                              148620421473601598,148620422591946835,148510422282863495,148510422530155570,
                              148590422977951649,148590423236157289]

In [3]:
table = PS1_detections(tenHighFidelityStarsNearby)

In [18]:
def Mean_mag(table):
    df = pd.DataFrame(columns = ['objID','ra','dec','gMeanPSFMag','gMeanPSFMagErr',
                    'rMeanPSFMag','rMeanPSFMagErr','iMeanPSFMag','iMeanPSFMagErr',
                    'zMeanPSFMag','zMeanPSFMagErr','yMeanPSFMag','yMeanPSFMagErr',
                    'gMeanApMag','gMeanApMagErr','rMeanApMag','rMeanApMagErr',
                    'iMeanApMag','iMeanApMagErr','zMeanApMag','zMeanApMagErr',
                    'yMeanApMag','yMeanApMagErr'])
    ids = table['objID'].unique()
    for i in ids:
        obj = table.iloc[table['objID'].values == i]
        filts = table['filter'].unique()
        obj_df = pd.DataFrame(data = np.zeros((23,23))*np.nan,
                    columns = ['objID','ra','dec','gMeanPSFMag','gMeanPSFMagErr',
                    'rMeanPSFMag','rMeanPSFMagErr','iMeanPSFMag','iMeanPSFMagErr',
                    'zMeanPSFMag','zMeanPSFMagErr','yMeanPSFMag','yMeanPSFMagErr',
                    'gMeanApMag','gMeanApMagErr','rMeanApMag','rMeanApMagErr',
                    'iMeanApMag','iMeanApMagErr','zMeanApMag','zMeanApMagErr',
                    'yMeanApMag','yMeanApMagErr'])
        obj_df = obj_df.iloc[0] # very gross "solution" to weird size error
        obj_df['objID'] = int(i)
        obj_df['ra'] = np.nanmedian(obj['ra'].values)
        obj_df['dec'] = np.nanmedian(obj['dec'].values)

        for f in filts:
            detections = obj.iloc[obj['filter'].values == f]
            psf_mag = -2.5 * np.log10(detections.psfFlux.values) + 8.9
            psf_mag_err = (-2.5 / np.log10(10)) * (detections.psfFluxErr.values 
                                                   / detections.psfFlux.values)
            psf_mag_err = np.sqrt(psf_mag_err**2 + 0.012**2)
            psf_mean, psf_err = Mean_calc(psf_mag,psf_mag_err)

            ap_mag = -2.5 * np.log10(detections.apFlux.values) + 8.9
            ap_mag_err = (-2.5 / np.log10(10)) * (detections.apFluxErr.values 
                                                  / detections.apFlux.values)
            ap_mag_err = np.sqrt(ap_mag_err**2 + 0.012**2)
            ap_mean, ap_err = Mean_calc(ap_mag,ap_mag_err)

            obj_df[f + 'MeanPSFMag'] = psf_mean
            obj_df[f + 'MeanPSFMagErr'] = psf_err

            obj_df[f + 'MeanApMag'] = ap_mean
            obj_df[f + 'MeanApMagErr'] = ap_err

        df = df.append(obj_df)
    return df

In [19]:
means = Mean_mag(table)

In [20]:
means

,objID,ra,dec,gMeanPSFMag,gMeanPSFMagErr,rMeanPSFMag,rMeanPSFMagErr,iMeanPSFMag,iMeanPSFMagErr,zMeanPSFMag,...,gMeanApMag,gMeanApMagErr,rMeanApMag,rMeanApMagErr,iMeanApMag,iMeanApMagErr,zMeanApMag,zMeanApMagErr,yMeanApMag,yMeanApMagErr
0,1.482804e+17,42.201680,33.568987,17.769360,0.007691,17.031250,0.004974,16.736575,0.004287,16.622044,...,17.755586,0.006767,17.040428,0.004730,16.735006,0.003974,16.632394,0.005640,16.544612,0.008931
0,1.486204e+17,42.037428,33.851985,16.475463,0.004483,15.664189,0.004317,15.334505,0.003312,15.192358,...,16.471525,0.004904,15.654562,0.004302,15.327676,0.003299,15.189320,0.004025,15.079544,0.005041
0,1.486204e+17,42.060013,33.856020,17.611481,0.007413,17.027492,0.004882,16.805493,0.004364,16.712664,...,17.613605,0.006485,17.024925,0.004715,16.796375,0.004034,16.711316,0.005089,16.633720,0.007111
0,1.486204e+17,42.090303,33.857621,16.239781,0.004643,15.004065,0.003721,14.148983,0.003736,13.758424,...,16.243618,0.004490,14.999904,0.004083,14.136608,0.003743,13.755485,0.003790,13.549586,0.003488
0,1.486204e+17,42.147392,33.850830,16.777901,0.006143,15.993192,0.004298,15.686259,0.003560,15.545651,...,16.773234,0.005830,15.986299,0.004261,15.684463,0.003644,15.543480,0.004429,15.437740,0.004725
0,1.486204e+17,42.259276,33.855144,15.545570,0.004694,15.145731,0.003076,14.982231,0.003034,14.940091,...,15.524692,0.004805,15.136398,0.003255,14.979103,0.003036,14.936737,0.005507,14.909186,0.004416
0,1.485104e+17,42.228323,33.760715,14.468125,0.004783,14.121092,0.003921,13.988046,0.003562,13.962969,...,14.462797,0.005155,14.105042,0.004386,13.976013,0.003726,13.951865,0.004227,13.926653,0.004241
0,1.485104e+17,42.253048,33.762460,15.421636,0.004260,14.993077,0.003710,14.838698,0.004812,14.780336,...,15.406368,0.004480,14.982764,0.003719,14.824162,0.004828,14.778255,0.005036,14.739198,0.004615
0,1.485904e+17,42.297831,33.825852,17.699002,0.007519,17.369146,0.005498,17.266185,0.005150,17.236521,...,17.680504,0.006627,17.376234,0.004942,17.259870,0.004979,17.240473,0.007362,17.204314,0.009939
0,1.485904e+17,42.323640,33.830553,16.397479,0.006346,15.434111,0.004052,15.019795,0.003727,14.822744,...,16.395199,0.006270,15.418338,0.004692,15.008386,0.003889,14.813080,0.005047,14.677613,0.004788


In [31]:
df = pd.DataFrame(data = np.zeros((23,23))*np.nan,columns = ['objID','ra','dec','gMeanPSFMag','gMeanPSFMagErr',
					'rMeanPSFMag','rMeanPSFMagErr','iMeanPSFMag','iMeanPSFMagErr',
					'zMeanPSFMag','zMeanPSFMagErr','yMeanPSFMag','yMeanPSFMagErr',
					'gMeanApMag','gMeanApMagErr','rMeanApMag','rMeanApMagErr',
					'iMeanApMag','iMeanApMagErr','zMeanApMag','zMeanApMagErr',
					'yMeanApMag','yMeanApMagErr'])
df = df.iloc[0]

In [4]:
table['objID'].unique()

array([148280422016753413, 148620420374152983, 148620420599947827,
       148620420902559763, 148620421473601598, 148620422591946835,
       148510422282863495, 148510422530155570, 148590422977951649,
       148590423236157289])

In [22]:
np.nanmin((table['psfFluxErr'] / table['psfFlux']) * (-2.5*np.log10(table['psfFluxErr'])+8.9))

0.02373856012163693

In [28]:
df.objID = 1

In [29]:
df

objID             1.0
ra                NaN
dec               NaN
gMeanPSFMag       NaN
gMeanPSFMagErr    NaN
rMeanPSFMag       NaN
rMeanPSFMagErr    NaN
iMeanPSFMag       NaN
iMeanPSFMagErr    NaN
zMeanPSFMag       NaN
zMeanPSFMagErr    NaN
yMeanPSFMag       NaN
yMeanPSFMagErr    NaN
Name: 0, dtype: float64

In [ ]:
pd.concat(df.)